# *MunchXML* Alpha
Developed by research assistant Loke Sjølie for the University of Oslo
___
Users:
I ask that you do not touch anything under the header **Program** unless you *sort of* know what you're doing. :)

Developers:
I apologize in advance.

In [1]:
version = "0.0" # Describes the "program's" state of completion and versioning.
uuid = "a403c593-09df-4538-8acf-8d459339fca8" # Unique ID. Used in sourceDesc of CMIF. Don't change it without a good reason.

*Vivisection of the TEI file provided on the 30th of March, 2022.*

*The file begins with a TEI statement. There is then a **single** teiheader tag with metadata for the whole file. In the primary directory there is then a text tag, followed by a body tag. All content is encapsulated within this body tag.*
1. The primary content vector is a set of multiple **div**s. Each div has an **ID**: div xml:id="No-MM_N1708".
2. Under the div is a head containing the ID.
3. Under the div are **multiple** lists. These lists have *items* that convey information.

## Program

### Init, metadata, etc

In [2]:
from bs4 import BeautifulSoup # Hent BeautifulSoup-modulen (https://www.crummy.com/software/BeautifulSoup/) for XML
from bs4 import Comment # BS4-addon for å håndtere kommentarer <!-- X -->
from datetime import date # Dates
#import glob # The yeast of thought and mind
#import os # Filsystem for lasting og lagring
today = date.today() # Sett dato i dag
today = today.strftime("%Y-%m-%d") # Formater dato
currVer = version+" "+today

In [3]:
previouslyRun = "Last executed code was version "+str(version)+" on date "+str(today)+". All OUTPUT files are current to that version on that date. UUID "+str(uuid)+"."
print("Version",currVer)
try:
    with open(r"settings/previous", "r", encoding="utf-8") as file: # Åpne filen med encoding UTF8
        f = file.readlines() # Les innholdet som linjer
        f = "".join(f) # Linjene blir kombinert i en variabel
    if f == previouslyRun:
        print("> Re-run detected. If you've changed anything, make sure to update the version var.")
    else:
        f = open(r"settings/previous", "w", encoding="utf-8")
        f.write(previouslyRun)
        print("> Updated internal files to reflect the date and version we're running today.")
    file.close()
except:
    f = open(r"settings/previous", "w", encoding="utf-8")
    f.write(previouslyRun)
    f.close()
    print("> I made a new file to keep track of when the script was last run.")

Version 0.0 2022-08-09
> Updated internal files to reflect the date and version we're running today.


### TEI-XML

In [7]:
target = r"INPUT/tei.xml"
with open(target, "r", encoding="utf-8") as file:
    tei = file.readlines() # Les innholdet som linjer
    tei = "".join(tei) # Linjene blir kombinert i en variabel
soup = BeautifulSoup(tei, from_encoding="UTF-8")

#### Debug stuff
What is this thing? An investigation of the input document's tags.

In [67]:
tagsAttrs = []
for x in soup.findAll():
    text = str(x.name)
    for i in x.attrs:
        text = text+" @"+i
    if text not in tagsAttrs:
        tagsAttrs.append(text)
dict = {
    "tei":"The TEI element - is where our file actually begins.",
    "teiheader":"The TEI header contains metadata (titleStmt, publicationStmt, sourceDesc...).",
    "p":"P is a paragraph. This is used in the TEIheader to contain the actual strings for publication & source desc. And a single, random </p> element later.",
    "body":"Body is used as a sub-element of <text> to contain all the metadata for all letters. I am personally offended by this practice. BS4 adds one, too.",
    "text":"Text appears to be a wrapper for the body tag, which contains all the texts' metadata.",
    "div":"Div, with @xml:id, is used to contain the metadata of a single letter.",
    "date":"Date is a date element. It seems to have the @when attribute very often, as well as enclosed text. Often has @type(year/fromTo, etc.)",
    "table":"Table is the primary data structure in which information about each letter is stored. This is a *table*.",
    "row":"Row is a sub-element of the table element. It defines a new X-axis in a table.",
    "seg":"Seg appears to be some kind of ID attached to each letter. The ID is used as an @xml:id attribute in div, and the element appears in references to other letters.",
    "cell":"Cell is a sub-element of the table element also. A single cell appears to be an entry into a row element.",
    "ref":"Ref appears to contain references to other XML items.",
    "item":"Item is a generic element that has multiple @attributes, such as owner, owner signature, author, paper type... This is evidently a very important tag.",
    "list":"List is a list. Often, the list only has one item. The list is used as description tag, containing other lists, and describes anything between dates to material type. Sigh...",
    "html":"The HTML tag can be ignored. BS4 adds this.",
    "filedesc":"Filedesc contains title, publication, source statements.",
    "sourcedesc":"Sourcedesc describes the source of the whole document.",
    "publicationstmt":"Publication statement for the whole document.",
    "title":"Title for the whole document.",
    "titlestmt":"Titlestmt is a wrapper for the title tag (whole document).",
    "div @xml:id":"Div has an attribute @xml:id. This describes the unique ID of the item in question.",
    "list @type":"List's @type attribute describes whether the list is wrapped around an object/physical description, a date, or other category.",
    "item @n":"Item's @n attribute describes role, library sorting, language, measures, dated, notes and so on. Very... multipurpose.",
    "tei @xmlns @xml:id":"tei @xmlns @xml:id is functionally identical to TEI tag. Just the one.",
    "date @type @from @to":"date @type @from @to describes the sequence type=fromTo, from, to. A date range.",
    "ref @target":"ref's @target attribute describes a URL to another XML.",
    "date @type @when":"Date with attributes type and when. Single date/year.",
    "ref @type @target":"Seems to contain URL to eMunch's web pages for a 'Read More' function.",
    "date @type @from":"date @type @from is an open-ended date.",
    "date @type":"Caution: date @type is a date with just a type. The date itself might be enclosed...? Potentially misleading. Investigate.",
    "ref @target @n":"ref @target @n - like ref @target, but @n tends to be the name of an institution or so.",
    "row @n":"row @n describes parts of the text. Inventory number, paper type, etc.",
    "ref @type @target @n":"ref @type @target @n - Working off of previous information, I'll infer that ref @type @target @n describes a Read More, with URL, with name."
    
}
print("Listing all unique tags and attribute combinations found with mapped, understood meanings.\n")
for x in tagsAttrs:
    if x in dict:
        if "@" in x:
            print("ATTR",dict[x])
        else:
            print("TAG",dict[x])
    else:
        print("\n"+str(x),"has no description. What is this?")
comments = soup.find_all(string=lambda text: isinstance(text, Comment))
if comments:
    n = len(comments)
    print("\n> Detected",n,"comments (<!-- -->, etc). These should be eradicated before tag extraction.")
else:
    print("\n> There are no (0) comments to worry about in this document.")
print("\n> You have... you've put an item in a list in an item to describe single years... *why*?")

Listing all unique tags found with mapped, understood meanings.
TAG The HTML tag can be ignored. BS4 adds this.
TAG Body is used as a sub-element of <text> to contain all the metadata for all letters. I am personally offended by this practice. BS4 adds one, too.
ATTR tei @xmlns @xml:id is functionally identical to TEI tag. Just the one.
TAG The TEI header contains metadata (titleStmt, publicationStmt, sourceDesc...).
TAG Filedesc contains title, publication, source statements.
TAG Titlestmt is a wrapper for the title tag (whole document).
TAG Title for the whole document.
TAG Publication statement for the whole document.
TAG P is a paragraph. This is used in the TEIheader to contain the actual strings for publication & source desc. And a single, random </p> element later.
TAG Sourcedesc describes the source of the whole document.
TAG Text appears to be a wrapper for the body tag, which contains all the texts' metadata.
ATTR Div has an attribute @xml:id. This describes the unique ID of 

In [60]:

tagsAttrs

['html',
 'body',
 'tei @xmlns @xml:id',
 'teiheader',
 'filedesc',
 'titlestmt',
 'title',
 'publicationstmt',
 'p',
 'sourcedesc',
 'text',
 'div @xml:id',
 'list @type',
 'item @n',
 'item',
 'date @type @from @to',
 'table',
 'row',
 'cell',
 'seg',
 'date @type @when',
 'ref @target',
 'list',
 'row @n',
 'ref @target @n',
 'div',
 'date',
 'ref @type @target',
 'date @type',
 'date @type @from',
 'ref @type @target @n']

### CMIF

https://correspsearch.net/en/documentation.html

/correspAction/@type == correspAction element with attribute type="xyz"

/correspAction/persName == correspAction element with persName child element

@X == attribute of element

*Each letter, postcard - document - that is to be described features its own **correspDesc element**. *There are as many correspDescs as there are items. A particular correspDesc element in CMI format is more restrictive and reduced with regard to its vocabulary than the TEI Guidlines generally allow. This enables interchange between the respective TEI documents.*

for each in letters:
    create correspDesc wrapper
    
<correspDesc>
    <correspAction type="sent">
        <persName ref="VIAFetc url">NAME</>
        <placeName ref="Geonames url">NAME</>
    <correspAction type="received">
        <persName ref="url">NAME</>
        <placeName ref="Geonames url">NAME</>